In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import pandas as pd
import requests
import os
from urllib import request, error
import urllib
import imageio
import cv2
import numpy as np
from collections import Counter
from PIL import Image
import gzip

### Creating csv that has top 16 landmarks

In [2]:
train = pd.read_csv('train.csv')
train = train[train['landmark_id'] != 'None']
train['landmark_id'] = pd.to_numeric(train['landmark_id'])
print('Original shape: ', train.shape)
landmark_counts = np.array(Counter(train['landmark_id']).most_common())
landmark_counts = landmark_counts.astype(np.int)
print('Number of landmarks: ', landmark_counts.shape)
landmark_counts = landmark_counts[landmark_counts[:,1] >= 6000] #6000 has 16 classes
print('Landmark counts: ', landmark_counts)
print('Number of images with 6000 per class: ', len(landmark_counts))

Original shape:  (1193114, 3)
Number of landmarks:  (14947, 2)
[[ 9633 49525]
 [ 6051 49306]
 [ 6599 22811]
 [ 9779 18096]
 [ 2061 13020]
 [ 5554 10819]
 [ 6651  9307]
 [ 5376  9061]
 [ 6696  9050]
 [ 4352  8839]
 [ 2743  8838]
 [13526  8525]
 [ 1553  7664]
 [10900  6838]
 [ 8063  6496]
 [ 8429  6329]]


In [19]:
allTrain = pd.DataFrame()
for i in range(len(landmark_counts)):
    landmark_index = np.where(train['landmark_id'] == landmark_counts[i][0])[0]
    print('Landmark: ', landmark_counts[i][0])
#     for j in range(2000): # I divided getting the images into 2 sessions: 32,000 the first time & 32,000 the second
    for j in range(2000, 4000):
        allTrain = allTrain.append(train.iloc[[landmark_index[j]]])

print('New shape: ', allTrain.shape)

# to csv
allTrain.to_csv('allTrain2000+2000.csv', encoding='utf-8', index=False)
allTrain = allTrain.reset_index(drop=True)

Landmark:  9633
Landmark:  6051
Landmark:  6599
Landmark:  9779
Landmark:  2061
Landmark:  5554
Landmark:  6651
Landmark:  5376
Landmark:  6696
Landmark:  4352
Landmark:  2743
Landmark:  13526
Landmark:  1553
Landmark:  10900
Landmark:  8063
Landmark:  8429
New shape:  (32000, 3)


In [3]:
allTrain = pd.read_csv('allTrain2000+2000.csv')
print(allTrain.shape)

(32000, 3)


### Creating image & label arrays

In [4]:
landmark_id = []
images = []
invalid = 0
t0 = time.time()
for i in range(len(allTrain)):
    if i % 100 == 0 and i != 0:
        print(i) # just to check progress
    try:
        request.urlopen(allTrain['url'][i])
    except:
        invalid += 1
        continue
    try:
        resp = urllib.request.urlopen(allTrain['url'][i])
        arr = np.asarray(bytearray(resp.read()), dtype="uint8")
        arr = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        arr = cv2.cvtColor(arr , cv2.COLOR_BGR2RGB)
        arr = cv2.resize(arr,(64, 64), interpolation=cv2.INTER_AREA)
    except:
        invalid += 1
        continue
    arr = np.transpose(arr, (2, 0, 1))
    landmark_id.append(int(allTrain['landmark_id'][i]))
    images.append(arr)
t1 = time.time()
print('Took: ', int(t1 - t0), ' seconds')
print('Number of invalid URLs: ', invalid)
landmark_id = np.array(landmark_id)
images = np.array(images)
images = images / 255
images = (images - 0.5) / 0.5
print(images.shape)

f = gzip.GzipFile("images64+2000.npy.gz", "w") # this is saving the 2nd batch of 32,000 images as the first batch had already been saved
np.save(file=f, arr=images)
f.close()

f = gzip.GzipFile("landmark_id64+2000.npy.gz", "w")
np.save(file=f, arr=landmark_id)
f.close()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

<!-- There are 6850 invalid URLs for top 16 classes (6000 images per class, 32x32): 6850 <br> -->
<!-- There are 1154 invalid URLs for top 16 classes (1000 images per class, 128x128): 1154 <br> -->
There are 2764 invalid URLs for top 16 classes (2000 images per class, 64x64, the first 32,000 images): 2764 <br>
There are 2340 invalid URLs for top 16 classes (2000 images per class, 64x64, the next 32,000 images): 2340 <br>

In [4]:
images = np.load('images64.npy')
landmark_id = np.load('landmark_id64.npy')

# converting the npy files to gz files to save memory space
f = gzip.GzipFile("images64.npy.gz", "w")
np.save(file=f, arr=images)
f.close()

f = gzip.GzipFile("landmark_id64.npy.gz", "w")
np.save(file=f, arr=landmark_id)
f.close()

# combining the two batches of 32,000 images into one file
f = gzip.open('images64.npy.gz','r')
images = np.array(np.load(f))
f = gzip.open('images64+2000.npy.gz','r')
images2000 = np.array(np.load(f))
images = np.append(images, images2000, axis=0)
images = np.array(images)

f = gzip.open('landmark_id64.npy.gz','r')
landmark_id = np.array(np.load(f))
f = gzip.open('landmark_id64+2000.npy.gz','r')
landmark_id2000 = np.array(np.load(f))
landmark_id = np.append(landmark_id, landmark_id2000, axis=0)
landmark_id = np.array(landmark_id)

print(images.shape)
print(landmark_id.shape)

# # saving the gz files that contain 64,000 images
f = gzip.GzipFile("landmark_id64All.npy.gz", "w")
np.save(file=f, arr=landmark_id)
f.close()

f = gzip.GzipFile("images64All.npy.gz", "w")
np.save(file=f, arr=landmark_id)
f.close()

(58896, 3, 64, 64)
(58896,)
